# Python II - Assignment 2

This **Home Assignment** is to be submitted and you will be given points for each of the tasks. It familiarizes you with basics of *web scraping* and basics of *regular expressions*.

## Formalities
**Submit in a group of 2-3 people until 22.06.2020 23:59CET. The deadline is strict!**

## Evaluation and Grading
General advice for programming excercises at *CSSH*:
Evaluation of your submission is done semi automatically. Think of it as this notebook being 
executed once. Afterwards, some test functions are appended to this file and executed respectively.

Therefore:
* Submit valid _Python3_ code only!
* Use external libraries only when specified by task.
* Ensure your definitions (functions, classes, methods, variables) follow the specification if
  given. The concrete signature of e.g. a function usually can be inferred from task description, 
  code skeletons and test cases.
* Ensure the notebook does not rely on current notebook or system state!
  * Use `Kernel --> Restart & Run All` to see if you are using any definitions, variables etc. that 
    are not in scope anymore.
  * Double check if your code relies on presence of files or directories other than those mentioned
    in given tasks. Tests run under Linux, hence don't use Windows style paths 
    (`some\path`, `C:\another\path`). Also, use paths only that are relative to and within your
    working directory (OK: `some/path`, `./some/path`; NOT OK: `/home/alice/python`, 
    `../../python`).
* Keep your code idempotent! Running it or parts of it multiple times must not yield different
  results. Minimize usage of global variables.
* Ensure your code / notebook terminates in reasonable time.

**There's a story behind each of these points! Don't expect us to fix your stuff!**

Regarding the scores, you will get no points for a task if:
- your function throws an unexpected error (e.g. takes the wrong number of arguments)
- gets stuck in an infinite loop
- takes much much longer than expected (e.g. >1s to compute the mean of two numbers)
- does not produce the desired output (e.g. returns an descendingly sorted list even though we asked for ascending, returns the mean and the std even though we asked for the mean only, only prints the output instead of returning it!)
- ...

# History in Wikipedia (10 points total)

Wikipedia has a lot of information on historic events. Assume you want to conduct a study that examines which language edition talks more about different historic event (as indicated by years). As an example you first consider the article "History_of_Germany" in the englisch and german wikipedia.

To get articles from the web, use the `requests` library. To deal with html content you can use the Beautiful soup library.

In [5]:
import requests
from bs4 import BeautifulSoup

## a) Grabbing a wikipedia article (1 + 0.5 + 0.5 + 0.5)
Write a function `get_article_from_web(article_name, language_edition)` that returns the HTML as string for that article in a specific language edition. Assume that the article exists.

To save bandwith when conducting multiple experiments we want to setup a cache of wikipedia articles.
To setup the cache write the function `save_article_to_disk(article_name, language_edition, content)` that saves the content for that wikipedia article in `'./cache/{language_edition}/{article_name}.html'`. If any of the folders do not exist they are created. Please read the information on evaluation and grading.


Then write a function `get_article_from_disk(article_name, language_edition)` that returns the cached version of the article from disk. If the article does not exists, it raises a `ValueError`.


Write a function `get_article(article_name, language_edition)` that uses a local cache for fetching articles from wikipedia. So if you that article exists in cache it returns the cached version. If not it fetches the html for that article from the web and writes it to the cache so there is no need to get it from the web the next time. Thereby use the previously defined functions.

Use the "caching" version `get_article` for all of the following tasks.

In [12]:
def get_article_from_web(article_name, language_edition):
    URL = f"https://{language_edition}.wikipedia.org/w/api.php"
    PARAMS = {
        "action": "parse",
        "page": "Python",
        "prop": "text",
        "section": 0,
        "format": "json"
        }
    results = requests.get(url=URL, params=PARAMS).json()
    soup = BeautifulSoup(results['parse']['text']['*'], 'html.parser')
    return soup.prettify()

In [13]:
print(get_article_from_web("Python","de"))

<div class="mw-parser-output">
 <p>
  <b>
   Python
  </b>
  (
  <a href="/wiki/Altgriechische_Sprache" title="Altgriechische Sprache">
   griechisch
  </a>
  <span class="Grek" lang="grc-Grek">
   Πύθων
  </span>
  ) steht für:
 </p>
 <ul>
  <li>
   <a href="/wiki/Python_(Mythologie)" title="Python (Mythologie)">
    Python (Mythologie)
   </a>
   , einen Drachen in der griechischen Mythologie
  </li>
  <li>
   <a href="/wiki/Pythons" title="Pythons">
    Pythons
   </a>
   (Pythonidae), eine Familie großer Schlangen
   <ul>
    <li>
     <a href="/wiki/Eigentliche_Pythons" title="Eigentliche Pythons">
      Eigentliche Pythons
     </a>
     <i>
      (Python)
     </i>
     , eine Gattung aus der Familie der Pythons
    </li>
   </ul>
  </li>
  <li>
   <a href="/wiki/Python_(Programmiersprache)" title="Python (Programmiersprache)">
    Python (Programmiersprache)
   </a>
   , eine Programmiersprache
  </li>
  <li>
   <a href="/wiki/Monty_Python" title="Monty Python">
    Monty Pytho

In [17]:
def save_article_to_disk(article_name, language_edition, content):
    try:
        os.makedirs(f"./cache/{language_edition}")
    except OSError:
        pass
    
    my_file=open(f"./cache/{language_edition}/{article_name}.html","w", encoding="utf-8")
    my_file.write(content)
    my_file.close()


In [18]:
content=get_article_from_web("Python","de")
save_article_to_disk("Python", "de", content)

In [24]:
def get_article_from_disk(article_name, language_edition):
    try:
        with open(f"./cache/{language_edition}/{article_name}.html","r",encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError: 
        raise ValueError("The article does not exists")



In [25]:
def get_article(article_name, language_edition):
    try:
        return get_article_from_disk(article_name,language_edition)
    except:
        content=get_article_from_web(article_name,language_edition)
        save_article_to_disk(article_name,language_edition,content)
        return get_article_from_disk(article_name,language_edition)

In [26]:
get_article("Germany","en")

'<div class="mw-parser-output">\n <table class="mbox-small plainlinks sistersitebox" role="presentation" style="background-color:#f9f9f9;border:1px solid #aaa;color:#000">\n  <tbody>\n   <tr>\n    <td class="mbox-image">\n     <img alt="" class="noviewer" data-file-height="512" data-file-width="512" decoding="async" height="40" src="//upload.wikimedia.org/wikipedia/commons/thumb/9/99/Wiktionary-logo-en-v2.svg/40px-Wiktionary-logo-en-v2.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/9/99/Wiktionary-logo-en-v2.svg/60px-Wiktionary-logo-en-v2.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/9/99/Wiktionary-logo-en-v2.svg/80px-Wiktionary-logo-en-v2.svg.png 2x" width="40"/>\n    </td>\n    <td class="mbox-text plainlist">\n     Look up\n     <i>\n      <b>\n       <a class="extiw" href="https://en.wiktionary.org/wiki/Python" title="wiktionary:Python">\n        Python\n       </a>\n      </b>\n     </i>\n     or\n     <i>\n      <b>\n       <a class="extiw" href="

## b) Links from one article to other articles  (1)
Write a function `get_links(article_name, language_edition)` that returns a list of wikipedia article names.  These links are obtained throught the article specified by `article_name` (in the language_edition). Only include links in its 'content' div. Do not include links that you can get through the left navigation bar.

Sort the links in alphabetically increasing order.

## c) Getting the same article in a different language edition (1.0)
Write a funciton `switch_language(article_name, old_language_edition, new_language_edition)` that returns the name of the wikipedia article in the new language edition.

## d) Using regular expressions to extract years from a page content (1.5)
Write a function `extract_years(string_input)` that gets a string and returns how often a certain year number from 1000-2019 occurs. You can assume that each number between 1000 and 2020 is a year number. The result is a dictionary with year numbers as keys, and the number of occurrences as values.

Example: `extract_years("The king reigned from 1245 to 1268. He died in 1268. His favorite number was 12689")` should return the dictionary `{1245: 1, 1268:2}`

## e) Aggregate years for articles (0.5)
Write a function `extract_years_for_articles(article_names, language_edition)` that extracts the years counts for all the articles in that particular language edition and aggregates them into a single dictionary. This aggregated dictionary is returned.

## f) Bringing it all together (1.5)
Write a function `get_all_years(base_article, base_language_edition, n=None, target_language_edition=None)`

- Determines 'real' base article (if target_language edition is specified, the article in the target language edition is used.)
- extracts the first n links (all if n is None) from that 'real' base article
- aggregates the year counts across the base article **and** all the n articles that the base article links to.
- it then returns that dictionary

## g) Visualize and Interpret your results (1 + 1)
Visualize the numbers of occurrences for the article History_of_Germany in both englisch (en) and german (de) in a timeline. Use `n=20`. Show the results here in the notebook and also save it to 'timeline.png' in code. Make sure the plot has a legend, axis labels, ...

Describe the visualization, and give reasons for possible differences of the german and englisch timelines. Write that string to a file 'timeline.txt'